### Salish Sea Oil Spill Scenarios

This notebook describes oil particle tracking for spills scenarios in the Salish Sea developed in conjunction with the Canadian Department of Fisheries and Oceans (DFO). The code if for translating particle tracking from *ocean parcels* using the *Salish Sea Cast* grid into input forcing files for *Atlantis* for the *Atlantis* box grid.

In [ ]:
import sys
import os
import math
import xarray as xr
import geopandas as gpd
import pandas as pd
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from netCDF4 import Dataset
from shapely.geometry import Point
from pathlib import Path
from pprint import pprint
from parcels import AdvectionRK4, VectorField, Variable
from parcels import FieldSet, plotTrajectoriesFile, Variable, ScipyParticle, Field
import numpy as np
from datetime import timedelta
sys.path.append('/ocean/rlovindeer/Atlantis/ssam_oceanparcels/Parcels_Utils/particle_tracking/parcels/')
from util.seed_particles import get_particles, get_release_times

In [ ]:
file_id = int(input( ))
scenario = {1 : "5b_Turn_Point_Diluted_bitumen",
            2 : "6a_VancouverHarbour_BunkerC",
            3 : "7a_JohnsonStrait_BunkerC",}
print("\nScenario running  :", scenario[file_id], sep = " ")

## Creating particle movement through Ocean Parcels

In [ ]:
#Kernels
def WindAdvectionRK4(particle, fieldset, time):
    """Advection of particles using fourth-order Runge-Kutta integration.
     Function needs to be converted to Kernel object before execution"""
    if particle.beached == 0:
        wp = fieldset.wind_percentage ## this need to be add to the fieldset
        
        if wp > 0:

            (u1, v1) = fieldset.UV[time, particle.depth, particle.lat, particle.lon]
            u1 = u1 * wp
            v1 = v1 * wp
            lon1, lat1 = (particle.lon + u1*.5*particle.dt, particle.lat + v1*.5*particle.dt)
            
            (u2, v2) = fieldset.UVwind[time + .5 * particle.dt, particle.depth, lat1, lon1]
            u2 = u2 * wp
            v2 = v2 * wp
            lon2, lat2 = (particle.lon + u2*.5*particle.dt, particle.lat + v2*.5*particle.dt)
            
            (u3, v3) = fieldset.UVwind[time + .5 * particle.dt, particle.depth, lat2, lon2]
            u3 = u3 * wp
            v3 = v3 * wp
            lon3, lat3 = (particle.lon + u3*particle.dt, particle.lat + v3*particle.dt)
            
            (u4, v4) = fieldset.UVwind[time + particle.dt, particle.depth, lat3, lon3]
            u4 = u4 * wp
            v4 = v4 * wp            
            
            u_wind  = (u1 + 2*u2 + 2*u3 + u4) / 6. * particle.dt
            v_wind  = (v1 + 2*v2 + 2*v3 + v4) / 6. * particle.dt
            
            particle.lon += (u1 + 2*u2 + 2*u3 + u4) / 6. * particle.dt
            particle.lat += (v1 + 2*v2 + 2*v3 + v4) / 6. * particle.dt
            
            particle.beached = 2
            
def BeachTesting(particle, fieldset, time):
    """ Testing if particles are on land. if 'yes' particle will be removed"""
    if particle.beached == 2:
        (u, v) = fieldset.UV[time, particle.depth, particle.lat, particle.lon]
        #print(u, v)
        if u == 0 and v == 0:
            particle.beached = 1
        else:
            particle.beached = 0

def DeleteParticle(particle, fieldset, time):
    particle.delete()

def DecayParticle(particle, fieldset, time):
    dt = particle.dt
    field_decay_value = fieldset.decay
    decay = math.exp(-1.0 * dt/field_decay_value)
    particle.decay_value = particle.decay_value * decay

In [ ]:
# Data Paths
currents = Path('/ocean/rlovindeer/Atlantis/Physics/Raw_Transport_Data/')
winds = Path('/ocean/rlovindeer/Atlantis/Physics/Wind/')
sea_grid = Path('/ocean/rlovindeer/Atlantis/Physics/Grids/ubcSSnBathymetryV17-02_a29d_efc9_4047.nc')
air_grid = Path('/ocean/rlovindeer/Atlantis/Physics/Grids/ubcSSaAtmosphereGridV1_0f03_6268_df4b.nc')

In [ ]:
# Salish Sea NEMO Model Grid, Geo-location and Bathymetry, v17-02

# Currents
u_current = sorted([p for p in currents.glob('2018-01*URaw_variables.nc')])
v_current = sorted([p for p in currents.glob('2018-01*VRaw_variables.nc')])

filenames = {
    'U': {'lon': sea_grid,'lat': sea_grid,'data': u_current},
    'V': {'lon': sea_grid,'lat': sea_grid,'data': v_current}
            }

variables = {'U': 'uVelocity','V': 'vVelocity'}
dimensions = {'lon': 'longitude', 'lat': 'latitude', 'time': 'time'}
print('creating from_nemo')
fieldset = FieldSet.from_nemo(filenames, variables, dimensions, allow_time_extrapolation=True)
print('creating from_nemo done')

fieldset.add_constant('decay', 1.0 * 3600.0)
print('add_constant decay')

In [ ]:
# HRDPS, Salish Sea, Atmospheric Forcing Grid, Geo-location, v1"

wind_paths = sorted([p for p in winds.glob('*_Wind_variables.nc')])
wind_filenames = {'lon': os.fspath(air_grid),'lat': os.fspath(air_grid),'data': wind_paths}
wind_dimensions = {'lon': 'longitude', 'lat': 'latitude', 'time': 'time'}

pprint(wind_filenames)

In [ ]:
Uwind_field = Field.from_netcdf(wind_filenames, ('U_wind', 'u_wind'),
                                     wind_dimensions,
                                     fieldtype='U',
                                     allow_time_extrapolation=True,
                                     transpose=False,
                                     deferred_load=False)
Vwind_field = Field.from_netcdf(wind_filenames, ('V_wind', 'v_wind'),
                                     wind_dimensions,
                                     fieldtype='V',
                                     allow_time_extrapolation=True,
                                     transpose=False,
                                     deferred_load=False)

print('wind data loaded')

In [ ]:
# change longitude for the wind field
Uwind_field.grid.lon = Uwind_field.grid.lon - 360
Vwind_field.grid.lon = Vwind_field.grid.lon - 360

[x_min, x_max, y_min, y_max] = Uwind_field.grid.lonlat_minmax

Uwind_field.grid.lonlat_minmax = [x_min - 360, x_max - 360, y_min, y_max]
Vwind_field.grid.lonlat_minmax = [x_min - 360, x_max - 360, y_min, y_max]

## adding the wind field to the fieldset object
fieldset.add_field(Uwind_field)
fieldset.add_field(Vwind_field)
wind_field = VectorField('UVwind', Uwind_field,  Vwind_field)
fieldset.add_vector_field(wind_field)

In [ ]:
# wind_percentage
# We need to do a sensitivity analysis of the percetage of wind to be used here
wind_percentage = 1
fieldset.add_constant('wind_percentage', wind_percentage/100.0)

In [ ]:
class MyParticle(ScipyParticle):
    initial_time = -100
    decay_value = Variable('decay_value', dtype=np.float32, initial=1.0)
    beached = Variable('beached', dtype=np.int32, initial=0.)
    age = Variable('age', dtype=np.int32, initial=0.)

# Particle Features
num_particles_per_day = 100
feature_release_index = 0
input_shapefile_name = "/ocean/rlovindeer/Atlantis/ssam_oceanparcels/SalishSea/Shape_Scenarios/" + scenario[file_id] + ".shp"
release_depth = -0.1
release_start_time = '2018-01-01'  ## winter start on December, Summer Jul - Aug  ## ask Susan about when to do simulation
release_end_time = '2018-01-02'
release_start_time = np.datetime64(release_start_time)
release_end_time = np.datetime64(release_end_time)
time_origin = fieldset.U.grid.time_origin.time_origin

print('setting up particles')

[release_times, p, num_particles] = get_release_times(time_origin, num_particles_per_day, release_start_time, release_end_time)
pset = get_particles(fieldset, num_particles, input_shapefile_name, MyParticle, feature_release_index, release_times, release_depth)

print(pset)

# Building the kernels
decay_kernel = pset.Kernel(DecayParticle)
beaching_kernel = pset.Kernel(BeachTesting)
ForcingWind_kernel = pset.Kernel(WindAdvectionRK4)

# Adding to the main kernel
my_kernel = AdvectionRK4 + decay_kernel + ForcingWind_kernel + beaching_kernel

OP_output_file_name = scenario[file_id] + '_oil_disperse.nc'
print(OP_output_file_name)

In [ ]:
try:
    os.system('rm ' + OP_output_file_name)
except:
    pass

print('executing particle kernel')

In [ ]:
## Output properties
output_file = pset.ParticleFile(name= OP_output_file_name, outputdt = timedelta(minutes = 60))
pset.execute(my_kernel,                 # the kernel (which defines how particles move)
             runtime=timedelta(hours = 24*6),   # total length of the run
             dt = timedelta(minutes = 60),      # timestep of the kernel
             output_file = output_file)         # file name and the time step of the outputs
output_file.close()

plotTrajectoriesFile(OP_output_file_name);

print('particle trajectories completed')

## Parcing Ocean Parcels output into Atlantis input files


In [ ]:
shapefile_name = "/ocean/rlovindeer/Atlantis/ssam_oceanparcels/SalishSea/SalishSea_July172019_2/SalishSea_July172019.shp"
data_df = gpd.read_file(shapefile_name)

In [ ]:
numLayers = 5;
numSites = data_df.shape[0]
numTargetSites = numSites

outputDT = 60*60 #1 hour

stepsPerDay = int(86400.0/ outputDT);
numStepsPerDT = stepsPerDay;
numStepsPerDT = int(outputDT/3600.0)

debug = False

inputFileName = OP_output_file_name
pfile = xr.open_dataset(str(inputFileName), decode_cf=True)

In [ ]:
lon = np.ma.filled(pfile.variables['lon'], np.nan)
lat = np.ma.filled(pfile.variables['lat'], np.nan)
time = np.ma.filled(pfile.variables['time'], np.nan)
z = np.ma.filled(pfile.variables['z'], np.nan)
probs = np.ma.filled(pfile.variables['decay_value'], np.nan)

numParticles = lon.shape[0]

trackDates = [];

for i in range(0,numParticles):
    print(time[i][0])
    trackDates.append(time[i][0])

RDiff = max(trackDates) - min(trackDates)

minDate = np.datetime64("2018-01-01T00:30:00");
ts = pd.to_datetime(str(minDate))
d = ts.strftime('%Y-%m-%d %H:%M:%S')
print(d)

In [ ]:
numReleaseDays = 1;
numReleaseSteps = numReleaseDays * stepsPerDay;
trackLength = len(lon[0]);

print('trackLength = ' + str(trackLength))
print('numStepsPerDT = ' + str(numStepsPerDT))

numSteps = int(trackLength / numStepsPerDT)

In [ ]:
# Create the netcdf output file for Atlantis

netcdfFileName = "Atlantis_" + scenario[file_id] + "_dt_" + str(outputDT) + ".nc"

try:
    os.remove(netcdfFileName)
except:
    pass

ncfile = Dataset(netcdfFileName, "w", format="NETCDF4", clobber=True)

# Dimensions
time = ncfile.createDimension("t", None)
b = ncfile.createDimension("b", numTargetSites)
z = ncfile.createDimension("z", numLayers)

# Variables
times = ncfile.createVariable("time","f4",("t",))
oil = ncfile.createVariable("oil","f4",("t", "b",))
Naphthalene = ncfile.createVariable("Naphthalene","f4",("t", "b",))
Phenanthrene = ncfile.createVariable("Phenanthrene","f4",("t", "b",))
Pyrene = ncfile.createVariable("Pyrene","f4",("t", "b",))
Benzo = ncfile.createVariable("Benzo","f4",("t", "b",))

# Attributes
times.units = "seconds since " + d
times.dt = str(outputDT);
oil.units = "mgPAH/m^3"
oil.long_name = "Amount of PAH"

Naphthalene.units = "mgPAH/m^3"
Naphthalene.long_name = "Amount of Naphthalene"

Phenanthrene.units = "mgPAH/m^3"
Phenanthrene.long_name = "Amount of Phenanthrene"

Pyrene.units = "mgPAH/m^3"
Pyrene.long_name = "Amount of Pyrene"

Benzo.units = "mgPAH/m^3"
Benzo.long_name = "Amount of Benzo(a)pyrene"

# Populate variables with data
timeData = np.arange(0,(numSteps + numReleaseSteps)*outputDT,outputDT)
times[:] = timeData;

boxDispersal = np.zeros((numSteps + numReleaseSteps, numTargetSites));

In [ ]:
for partIndex in range(0, numParticles):

    trackDateDiff = trackDates[partIndex] - minDate;
    trackDateDiff = trackDateDiff/ np.timedelta64(1, 's')

    timeOffset = int(abs((trackDateDiff /outputDT)));

    for stepIndex in range(0, numSteps):
        timeValue = stepIndex + timeOffset

        partLon = lon[partIndex][stepIndex * numStepsPerDT];
        partLat = lat[partIndex][stepIndex * numStepsPerDT];
        partProb = probs[partIndex][stepIndex * numStepsPerDT];

        matchFound = 0;

        for targetIndex in range(0, numTargetSites):

            path = data_df.iloc[targetIndex].geometry
            checks = path.contains(Point(partLon, partLat));

            if checks:
                boxDispersal[timeValue][targetIndex] = boxDispersal[timeValue][targetIndex] + partProb;

                # uncomment line below to ignore particle decay during debugging.
                #boxDispersal[timeValue][targetIndex] = boxDispersal[timeValue][targetIndex] + 1.0

                matchFound = 1
                if debug:
                    print('At time ' + str(timeValue) + ' Particle (' + str(partIndex) + ') in box ' + str(data_df.iloc[targetIndex].BOX_ID))


                break;

        if matchFound == 0:
            if debug:
                print('No match for particle')
                print(partLon, partLat)


        #break

oil[:, :] = boxDispersal;
Naphthalene[:, :] = boxDispersal;
Phenanthrene[:, :] = boxDispersal;
Pyrene[:, :] = boxDispersal;
Benzo[:, :] = boxDispersal;

ncfile.close()

## Animating Oil Dispersal Scenario in the Salish Sea Atlantis Model

In [ ]:
boxes = data_df['BOTZ']
land_boxes = boxes==0
land_boxes = data_df.index[land_boxes]

numReleaseDays = RDiff
print('numReleaseDays = ' + str(numReleaseDays))

numReleaseDTS = int(abs(numReleaseDays/np.timedelta64(1, 'h')));
totalNumOfTS = int(numReleaseDTS + trackLength);
print('totalNumOfTS = ' + str(totalNumOfTS))
print('trackLength = ' + str(trackLength))
print(numParticles)

In [ ]:
trackLonsPadded = np.zeros((int(numParticles), totalNumOfTS));
trackLatsPadded = np.zeros((int(numParticles), totalNumOfTS));
particlesAge = np.zeros((int(numParticles), totalNumOfTS));

for trackIndex in range(0,numParticles):

    print(trackDates[trackIndex])
    print(minDate)
    trackDateDiff = trackDates[trackIndex] - minDate

    print(trackDateDiff/np.timedelta64(1, 'h'))

    trackNumsToPad = int(trackDateDiff/np.timedelta64(1, 'h'))
    print(trackNumsToPad)

    trackLonsPadded[trackIndex][0:trackNumsToPad] = 0;
    trackLatsPadded[trackIndex][0:trackNumsToPad] = 0;

    trackLonsPadded[trackIndex][trackNumsToPad:trackNumsToPad + trackLength] = lon[:][trackIndex];
    trackLatsPadded[trackIndex][trackNumsToPad:trackNumsToPad + trackLength] = lat[:][trackIndex];

numSteps = int(trackLength / numStepsPerDT)

In [ ]:
savefile_prefix = 'boxes'

pfile = xr.open_dataset(str(netcdfFileName), decode_cf=True)

print(pfile)

time = np.ma.filled(pfile.variables['time'], np.nan)

oil = np.ma.filled(pfile.variables['Benzo'], np.nan)
num_steps = time.shape[0]
print(num_steps)

In [ ]:
v_max = 80
_cmap = cm.viridis
file_names = []

land_df = data_df.loc[land_boxes]

for time_index in range(0, num_steps):

    plon = trackLonsPadded[:, time_index]
    plat = trackLatsPadded[:, time_index]

    plon = plon[plon<0]
    plat = plat[plat>0]
    time_oil = oil[time_index]

    data_df['oil'] = time_oil
    data_df.loc[land_boxes, 'oil'] = 10000

    ax = data_df.plot(figsize=(15, 15), column = 'oil', cmap = _cmap, vmin=0, vmax=v_max, legend=True)
    land_df.plot(ax=ax, color='gray')
    ax.scatter(plon, plat, s=10, color='red', zorder=20)

    ax.set_title(time[time_index])

    plot_name = savefile_prefix + '_time_' + str(time_index).zfill(3) + '.png'
    plt.savefig(plot_name)
    file_names.append(plot_name)
    plt.close()

In [ ]:
from PIL import Image
import glob
 
# Create the frames
frames = []
imgs = glob.glob("*.png")
imgs.sort() 
for i in imgs:
    new_frame = Image.open(i)
    frames.append(new_frame)

# Save into loop
frames[0].save('Oil_Scenario_' + scenario[file_id]+ '.gif', format='GIF',
               append_images=frames[1:],
               save_all=True,
               duration=300, loop=0)


In [ ]:
file_name_str = ' '.join(file_names)
os.system('rm ' + file_name_str)

In [ ]:
from IPython.display import Image
Image(filename='Oil_Scenario_' + scenario[file_id]+ '.gif') 